<img src="../assets/notebook_banner.png" alt="Notebook Banner" width= "100%">


# **Chinook Retail Analytics — SQL-Driven Business Insights**


Welcome to the **Chinook SQL Analytics Project Notebook**.  

This notebook contains a step-by-step, reproducible analysis of the Chinook digital music store dataset using **SQL within Jupyter Notebook**.  
The primary goal is to deliver **actionable, data-driven insights** that support strategic decision-making across marketing, sales, catalog curation, and team performance.  

---

## Notebook Objectives  

- **Customer & Market Insights** — Identify high-value customers, revenue by country, and average order value trends.  
- **Sales & Revenue Analytics** — Track total revenue, units sold, top tracks, artists, genres, and formats.  
- **Basket & Pricing Behavior** — Explore basket sizes, basket value distribution, and price sensitivity.  
- **Catalogue & Content Curation** — Detect catalogue gaps, playlist opportunities, and high-performing content.  
- **Team Performance (Account Managers)** — Measure rep-managed revenue, average invoice value, and portfolio coverage.  

---

## Notebook Structure  

This notebook is organized into the following sections:  

1. **Environment Setup** – Connecting Jupyter to the Chinook SQLite database.  
2. **Exploratory SQL Queries** – Writing and executing queries to answer business questions.  
3. **Insights & Recommendations** – Interpreting results for stakeholders and proposing actions.  
4. **Conclusion** – Summarizing key findings and business impact.  

---

<a id="cont"></a>

## 📚 Table of Contents

<a href=#one>1. Customer & Market Insights</a>

<a href=#two>2. Company's Sales & Revenue Analytics</a>

<a href=#three>3. Basket & Pricing Behavior</a>

<a href=#four>4. Catalogue & Content Curation</a>

<a href=#five>5. Team Performance</a>

## 🔧 Setup

In [ ]:
# Install required libraries (if not already installed)
%pip install sqlalchemy ipython-sql pandas matplotlib

In [ ]:
# Load Jupyter SQL extension
%load_ext sql

In [ ]:
# Connect to the Chinook database
%sql sqlite:///databa/chinook.db


<a id="one"></a>
## 1️⃣. Customer & Market Insights
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Business Problem from Management ⚡ |
| :--------------------------- |
| Management needs to understand all revenue sources. Placing priority on;   
- The most valuable customers, 
- Best regions/countries of sales, 
- Retainable customers and one-timer shoppers. 
- Market-level Average Order Value (AOV)  

---


### **Queries**

### 1.1 Company's Top Lifetime Value Customers

In [5]:
%%sql
SELECT c.CustomerId,
       c.FirstName || ' ' || c.LastName AS Customer,
       c.Country,
       ROUND(SUM(i.Total), 2) AS LifetimeRevenue
FROM customers c
JOIN invoices  i ON i.CustomerId = c.CustomerId
GROUP BY c.CustomerId
ORDER BY LifetimeRevenue DESC
LIMIT 20;

 * sqlite:///chinook.db
Done.


CustomerId,Customer,Country,LifetimeRevenue
6,Helena Holý,Czech Republic,49.62
26,Richard Cunningham,USA,47.62
57,Luis Rojas,Chile,46.62
45,Ladislav Kovács,Hungary,45.62
46,Hugh O'Reilly,Ireland,45.62
24,Frank Ralston,USA,43.62
28,Julia Barnett,USA,43.62
37,Fynn Zimmermann,Germany,43.62
7,Astrid Gruber,Austria,42.62
25,Victor Stevens,USA,42.62


### 1.2 Company's Revenue Generated by Country

In [6]:
%%sql
WITH revenue_by_country AS (
    SELECT 
        c.Country,
        ROUND(SUM(i.Total), 2) AS Revenue
    FROM customers c
    JOIN invoices i ON i.CustomerId = c.CustomerId
    GROUP BY c.Country
),
ranked AS (
    SELECT 
        Country,
        Revenue,
        ROUND((Revenue * 100.0 / (SELECT SUM(Revenue) FROM revenue_by_country)), 2) AS RevenuePercentage
    FROM revenue_by_country
    ORDER BY Revenue DESC
)
SELECT 
    Country,
    Revenue,
    RevenuePercentage,
    ROUND(SUM(RevenuePercentage) OVER (ORDER BY Revenue DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW), 2) AS CumulativePercentage
FROM ranked
ORDER BY Revenue DESC;


 * sqlite:///chinook.db
Done.


Country,Revenue,RevenuePercentage,CumulativePercentage
USA,523.06,22.46,22.46
Canada,303.96,13.05,35.51
France,195.1,8.38,43.89
Brazil,190.1,8.16,52.05
Germany,156.48,6.72,58.77
United Kingdom,112.86,4.85,63.62
Czech Republic,90.24,3.88,67.5
Portugal,77.24,3.32,70.82
India,75.26,3.23,74.05
Chile,46.62,2.0,76.05


### 1.3 Company's Average Order Value (AOV).

In [7]:
%%sql
SELECT c.Country,
       ROUND(AVG(i.Total), 2) AS AvgOrderValue
FROM customers c
JOIN invoices  i ON i.CustomerId = c.CustomerId
GROUP BY c.Country
ORDER BY AvgOrderValue DESC;

 * sqlite:///chinook.db
Done.


Country,AvgOrderValue
Chile,6.66
Ireland,6.52
Hungary,6.52
Czech Republic,6.45
Austria,6.09
Finland,5.95
Netherlands,5.8
India,5.79
USA,5.75
Norway,5.66


### 1.4  Returning  vs. One-timer Customers (Based on Number o Invoices)

In [7]:
%%sql
WITH counts AS (
  SELECT c.CustomerId, COUNT(i.InvoiceId) AS Orders
  FROM customers c
  LEFT JOIN invoices i ON i.CustomerId = c.CustomerId
  GROUP BY c.CustomerId
)
SELECT
  SUM(CASE WHEN Orders = 1 THEN 1 ELSE 0 END) AS OneTimeCustomers,
  SUM(CASE WHEN Orders > 1 THEN 1 ELSE 0 END) AS ReturningCustomers
FROM counts;

 * sqlite:///chinook.db
Done.


OneTimeCustomers,ReturningCustomers
0,59



<a id="two"></a>
## 2️⃣. Company's Sales & Revenue Analytics
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Business Problem from Management ⚡ |
| :--------------------------- |
| Management needs a single view of the overall sales performance: 
- Company's total revenue 
- Company's monthly revenue, and 
- Sales trends, and top-performing tracks, artists, genres and formats to steer promotions and targets.  

---


### **Queries**

### 2.1  Headline KPIs: revenue & units sold

In [25]:
%%sql
SELECT ROUND(SUM(i.Total), 2) AS TotalRevenue,
       SUM(ii.Quantity)       AS UnitsSold
FROM invoices i
JOIN invoice_items ii ON ii.InvoiceId = i.InvoiceId;

 * sqlite:///chinook.db
Done.


TotalRevenue,UnitsSold
20848.62,2240


### 2.2 Monthly revenue trend (Month-Year)

In [26]:
%%sql
SELECT 
  SUBSTR(i.InvoiceDate, 6, 2) || '-' || SUBSTR(i.InvoiceDate, 1, 4) AS MonthYear,
  ROUND(SUM(i.Total), 2) AS Revenue
FROM invoices i
GROUP BY SUBSTR(i.InvoiceDate, 6, 2) || '-' || SUBSTR(i.InvoiceDate, 1, 4)
ORDER BY MonthYear;

 * sqlite:///chinook.db
Done.


MonthYear,Revenue
01-2009,35.64
01-2010,52.62
01-2011,37.62
01-2012,37.62
01-2013,37.62
02-2009,37.62
02-2010,46.62
02-2011,37.62
02-2012,37.62
02-2013,27.72


###     2.3 Top 10 tracks by revenue

In [13]:
%%sql
SELECT t.TrackId,
       t.Name AS Track,
       ar.Name AS Artist,
       ROUND(SUM(ii.UnitPrice * ii.Quantity), 2) AS Revenue
FROM invoice_items ii
JOIN tracks   t  ON t.TrackId   = ii.TrackId
JOIN albums   al ON al.AlbumId  = t.AlbumId
JOIN artists  ar ON ar.ArtistId = al.ArtistId
GROUP BY t.TrackId
ORDER BY Revenue DESC
LIMIT 10;


 * sqlite:///chinook.db
Done.


TrackId,Track,Artist,Revenue
2832,The Woman King,Battlestar Galactica,3.98
2850,The Fix,Heroes,3.98
2868,Walkabout,Lost,3.98
3177,Hot Girl,The Office,3.98
3200,Gay Witch Hunt,The Office,3.98
3214,Phyllis's Wedding,The Office,3.98
3223,How to Stop an Exploding Man,Heroes,3.98
3250,Pilot,Aquaman,3.98
2820,Occupation / Precipice,Battlestar Galactica,1.99
2821,"Exodus, Pt. 1",Battlestar Galactica,1.99


###     2.4 Top 10 artists by revenue

In [10]:
%%sql
SELECT ar.ArtistId,
       ar.Name AS Artist,
       ROUND(SUM(ii.UnitPrice * ii.Quantity), 2) AS Revenue
FROM invoice_items ii
JOIN tracks  t  ON t.TrackId   = ii.TrackId
JOIN albums  al ON al.AlbumId  = t.AlbumId
JOIN artists ar ON ar.ArtistId = al.ArtistId
GROUP BY ar.ArtistId
ORDER BY Revenue DESC
LIMIT 10;

 * sqlite:///chinook.db
Done.


ArtistId,Artist,Revenue
90,Iron Maiden,138.6
150,U2,105.93
50,Metallica,90.09
22,Led Zeppelin,86.13
149,Lost,81.59
156,The Office,49.75
113,Os Paralamas Do Sucesso,44.55
58,Deep Purple,43.56
82,Faith No More,41.58
81,Eric Clapton,39.6


###     2.5 Revenue by genre

In [13]:
%%sql
SELECT g.Name AS Genre,
       ROUND(SUM(ii.UnitPrice * ii.Quantity), 2) AS Revenue
FROM invoice_items ii
JOIN tracks t  ON t.TrackId  = ii.TrackId
JOIN genres g  ON g.GenreId  = t.GenreId
GROUP BY g.GenreId
ORDER BY Revenue DESC;

 * sqlite:///chinook.db
Done.


Genre,Revenue
Rock,826.65
Latin,382.14
Metal,261.36
Alternative & Punk,241.56
TV Shows,93.53
Jazz,79.2
Blues,60.39
Drama,57.71
R&B/Soul,40.59
Classical,40.59


In [ ]:
%%sql
    WITH GenreRevenue AS (
        SELECT 
            g.Name AS Genre,
            ROUND(SUM(ii.UnitPrice * ii.Quantity), 2) AS Revenue
        FROM invoice_items ii
        JOIN tracks t ON t.TrackId = ii.TrackId
        JOIN genres g ON g.GenreId = t.GenreId
        GROUP BY g.GenreId
    ),
    TotalRevenue AS (
        SELECT SUM(Revenue) AS Total FROM GenreRevenue
    )
    SELECT 
        gr.Genre,
        gr.Revenue,
        ROUND(100.0 * gr.Revenue / tr.Total, 2) AS Percentage,
        ROUND(SUM(100.0 * gr.Revenue / tr.Total) OVER (ORDER BY gr.Revenue DESC), 2) AS Cumulative_Percentage
    FROM GenreRevenue gr, TotalRevenue tr
    ORDER BY gr.Revenue DESC;


 * sqlite:///chinook.db
Done.


Genre,Revenue,Percentage,Cumulative_Percentage
Rock,826.65,35.5,35.5
Latin,382.14,16.41,51.91
Metal,261.36,11.22,63.13
Alternative & Punk,241.56,10.37,73.51
TV Shows,93.53,4.02,77.52
Jazz,79.2,3.4,80.93
Blues,60.39,2.59,83.52
Drama,57.71,2.48,86.0
R&B/Soul,40.59,1.74,89.48
Classical,40.59,1.74,89.48


###     2.6 Revenue by media type (format)

In [14]:
%%sql
SELECT mt.Name AS MediaType,
       ROUND(SUM(ii.UnitPrice * ii.Quantity), 2) AS Revenue
FROM invoice_items ii
JOIN tracks t   ON t.TrackId         = ii.TrackId
JOIN media_types mt ON mt.MediaTypeId = t.MediaTypeId
GROUP BY mt.MediaTypeId
ORDER BY Revenue DESC;

 * sqlite:///chinook.db
Done.


MediaType,Revenue
MPEG audio file,1956.24
Protected MPEG-4 video file,220.89
Protected AAC audio file,144.54
Purchased AAC audio file,3.96
AAC audio file,2.97


In [32]:
%%sql
WITH MediaTypeRevenue AS (
    SELECT 
        mt.Name AS MediaType,
        ROUND(SUM(ii.UnitPrice * ii.Quantity), 2) AS Revenue
    FROM invoice_items ii
    JOIN tracks t ON t.TrackId = ii.TrackId
    JOIN media_types mt ON mt.MediaTypeId = t.MediaTypeId
    GROUP BY mt.MediaTypeId
),
TotalRevenue AS (
    SELECT SUM(Revenue) AS Total FROM MediaTypeRevenue
)
SELECT 
    mtr.MediaType,
    mtr.Revenue,
    ROUND(100.0 * mtr.Revenue / tr.Total, 2) AS Percentage
FROM MediaTypeRevenue mtr, TotalRevenue tr
ORDER BY mtr.Revenue DESC;


 * sqlite:///chinook.db
Done.


MediaType,Revenue,Percentage
MPEG audio file,1956.24,84.01
Protected MPEG-4 video file,220.89,9.49
Protected AAC audio file,144.54,6.21
Purchased AAC audio file,3.96,0.17
AAC audio file,2.97,0.13



<a id="three"></a>
## 3️⃣. Basket & Pricing Behavior
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Business Problem from Management ⚡ |
| :--------------------------- |
| Management wants to understand basket composition (size and value) and identify: 
- Which invoices are large? 
- Which products sell high volume on discount? 

---


### **Queries**

###     3.1 Largest baskets by quantity and value

In [37]:
%%sql
SELECT ii.InvoiceId,
       SUM(ii.Quantity) AS ItemsInBasket,
       ROUND(SUM(ii.UnitPrice * ii.Quantity), 2) AS BasketValue
FROM invoice_items ii
GROUP BY ii.InvoiceId
ORDER BY ItemsInBasket DESC, BasketValue DESC
LIMIT 60;

 * sqlite:///chinook.db
Done.


InvoiceId,ItemsInBasket,BasketValue
404,14,25.86
299,14,23.86
96,14,21.86
194,14,21.86
89,14,18.86
201,14,18.86
306,14,16.86
313,14,16.86
103,14,15.86
208,14,15.86


###     3.2 Popular tracks with lower average prices.

In [21]:
%%sql
WITH track_stats AS (
  SELECT 
    t.TrackId,
    t.Name,
    AVG(ii.UnitPrice) AS AvgPrice,
    SUM(ii.Quantity)  AS TotalQty,
    SUM(ii.UnitPrice * ii.Quantity) AS TotalRevenue
  FROM tracks t
  JOIN invoice_items ii ON ii.TrackId = t.TrackId
  GROUP BY t.TrackId
)
SELECT 
  Name, 
  ROUND(AvgPrice, 2) AS AvgPrice, 
  TotalQty,
  ROUND(TotalRevenue, 2) AS TotalRevenue
FROM track_stats
ORDER BY TotalQty DESC, AvgPrice ASC
LIMIT 15;


 * sqlite:///chinook.db
Done.


Name,AvgPrice,TotalQty,TotalRevenue
Balls to the Wall,0.99,2,1.98
Inject The Venom,0.99,2,1.98
Snowballed,0.99,2,1.98
Overdose,0.99,2,1.98
Deuces Are Wild,0.99,2,1.98
Not The Doctor,0.99,2,1.98
Por Causa De Você,0.99,2,1.98
Welcome Home (Sanitarium),0.99,2,1.98
Snowblind,0.99,2,1.98
Cornucopia,0.99,2,1.98



<a id="four"></a>
## 4️⃣. Catalogue & Content Curation
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Business Problem from Management ⚡ |
| :--------------------------- |
| Content team needs to ensure the catalogue is well curated by having insights on: 
- Which albums have depth?
- Where are curation gaps (tracks not in any playlist)? 
- What is the most play-listed (social proof). 

---


### **Queries**

### 4.1 Albums with deepest tracklists

In [ ]:
%%sql
SELECT al.AlbumId,
       al.Title AS Album,
       ar.Name  AS Artist,
       COUNT(t.TrackId) AS NumTracks
FROM albums al
JOIN artists ar ON ar.ArtistId = al.ArtistId
JOIN tracks  t  ON t.AlbumId   = al.AlbumId
GROUP BY al.AlbumId
ORDER BY NumTracks DESC, Album;

 * sqlite:///chinook.db
Done.


AlbumId,Album,Artist,NumTracks
141,Greatest Hits,Lenny Kravitz,57
23,Minha Historia,Chico Buarque,34
73,Unplugged,Eric Clapton,30
229,"Lost, Season 3",Lost,26
230,"Lost, Season 1",Lost,25
251,"The Office, Season 3",The Office,25
253,"Battlestar Galactica (Classic), Season 1",Battlestar Galactica (Classic),24
231,"Lost, Season 2",Lost,24
83,My Way: The Best Of Frank Sinatra [Disc 1],Frank Sinatra,24
24,Afrociberdelia,Chico Science & Nação Zumbi,23


### 4.2 Tracks missing from any playlist (curation gaps)

In [23]:
%%sql
SELECT t.TrackId, t.Name AS Track, ar.Name AS Artist
FROM tracks t
JOIN albums al  ON al.AlbumId  = t.AlbumId
JOIN artists ar ON ar.ArtistId = al.ArtistId
LEFT JOIN playlist_track pt ON pt.TrackId = t.TrackId
WHERE pt.PlaylistId IS NULL
ORDER BY Artist, Track
LIMIT 50;

 * sqlite:///chinook.db
Done.


TrackId,Track,Artist


### 4.3 Most playlisted tracks (social proof)

In [24]:
%%sql
SELECT t.Name AS Track,
       ar.Name AS Artist,
       COUNT(pt.PlaylistId) AS PlaylistsCount
FROM playlist_track pt
JOIN tracks t  ON t.TrackId  = pt.TrackId
JOIN albums al ON al.AlbumId = t.AlbumId
JOIN artists ar ON ar.ArtistId = al.ArtistId
GROUP BY t.TrackId
ORDER BY PlaylistsCount DESC, Track
LIMIT 20;

 * sqlite:///chinook.db
Done.


Track,Artist,PlaylistsCount
"A Midsummer Night's Dream, Op.61 Incidental Music: No.7 Notturno",Philharmonia Orchestra & Sir Neville Marriner,5
"Aria Mit 30 Veränderungen, BWV 988 ""Goldberg Variations"": Aria",Wilhelm Kempff,5
Ave Maria,"Britten Sinfonia, Ivor Bolton & Lesley Garrett",5
Carmen: Overture,"Chor der Wiener Staatsoper, Herbert Von Karajan & Wiener Philharmoniker",5
Carmina Burana: O Fortuna,Boston Symphony Orchestra & Seiji Ozawa,5
Cavalleria Rusticana \ Act \ Intermezzo Sinfonico,James Levine,5
"Concerto for Piano No. 2 in F Minor, Op. 21: II. Larghetto","Emanuel Ax, Eugene Ormandy & Philadelphia Orchestra",5
"Concerto for Violin, Strings and Continuo in G Major, Op. 3, No. 9: I. Allegro","Mela Tenenbaum, Pro Musica Prague & Richard Kapp",5
"Das Lied Von Der Erde, Von Der Jugend",Gustav Mahler,5
Die Walküre: The Ride of the Valkyries,Sir Georg Solti & Wiener Philharmoniker,5



<a id="five"></a>
## 5️⃣. Team Performance (Support Reps as Account Managers)
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Business Problem from Management ⚡ |
| :--------------------------- |
| Sales leadership needs to be able to measure employee's effectiveness and performance visibility:
- Revenue under management
- Average invoice value  
- Portfolio size to guide management decision on incentives and coverage

---


### **Queries**

### 5.1 Revenue managed by each sales representative

In [29]:
%%sql
SELECT e.EmployeeId,
       e.FirstName || ' ' || e.LastName AS Employee,
       ROUND(SUM(i.Total), 2) AS RevenueManaged
FROM employees e
JOIN customers c ON c.SupportRepId = e.EmployeeId
JOIN invoices  i ON i.CustomerId   = c.CustomerId
GROUP BY e.EmployeeId
ORDER BY RevenueManaged DESC;

 * sqlite:///chinook.db
Done.


EmployeeId,Employee,RevenueManaged
3,Jane Peacock,833.04
4,Margaret Park,775.4
5,Steve Johnson,720.16


### 5.2 Average invoice value per sales representative

In [30]:
%%sql
SELECT e.EmployeeId,
       e.FirstName || ' ' || e.LastName AS Employee,
       ROUND(AVG(i.Total), 2) AS AvgInvoice
FROM employees e
JOIN customers c ON c.SupportRepId = e.EmployeeId
JOIN invoices  i ON i.CustomerId   = c.CustomerId
GROUP BY e.EmployeeId
ORDER BY AvgInvoice DESC;

 * sqlite:///chinook.db
Done.


EmployeeId,Employee,AvgInvoice
5,Steve Johnson,5.72
3,Jane Peacock,5.71
4,Margaret Park,5.54


### 5.3. Portfolio size vs revenue

In [27]:
%%sql
SELECT e.EmployeeId,
       e.FirstName || ' ' || e.LastName AS Employee,
       COUNT(DISTINCT c.CustomerId) AS CustomersManaged,
       ROUND(SUM(i.Total), 2)       AS RevenueManaged
FROM employees e
LEFT JOIN customers c ON c.SupportRepId = e.EmployeeId
LEFT JOIN invoices  i ON i.CustomerId   = c.CustomerId
GROUP BY e.EmployeeId
ORDER BY RevenueManaged DESC, CustomersManaged DESC;

 * sqlite:///chinook.db
Done.


EmployeeId,Employee,CustomersManaged,RevenueManaged
3,Jane Peacock,21,833.04
4,Margaret Park,20,775.4
5,Steve Johnson,18,720.16
1,Andrew Adams,0,None
2,Nancy Edwards,0,None
6,Michael Mitchell,0,None
7,Robert King,0,None
8,Laura Callahan,0,None
